# RFM Analysis on Online Retail Data Set - With Monthly Aggregate View

In [1]:
import sqlite3
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
#print multiple statement in the same line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#load the csv into dataframe
orig_retail_df = pd.read_csv("C:\\DataSets\\OnlineRetail2Sample.csv")
orig_retail_df.head()

,InvoiceNo,StockCode,Quantity,DoP,UnitPrice,CustomerID,Country
0,536365,85123A,6,12/1/2010,2.55,17850,United Kingdom
1,536365,71053,6,12/1/2010,3.39,17850,United Kingdom
2,536365,84406B,8,12/1/2010,2.75,17850,United Kingdom
3,536365,84029G,6,12/1/2010,3.39,17850,United Kingdom
4,536365,84029E,6,12/1/2010,3.39,17850,United Kingdom


In [3]:
#check for missing values
pd.isnull(orig_retail_df).any()
pd.isnull(orig_retail_df).sum()

InvoiceNo     False
StockCode     False
Quantity      False
DoP           False
UnitPrice     False
CustomerID    False
Country       False
dtype: bool

InvoiceNo     0
StockCode     0
Quantity      0
DoP           0
UnitPrice     0
CustomerID    0
Country       0
dtype: int64

In [4]:
#create the table RFMDM_TEMP
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cursor = conn.cursor()
cursor.execute('''DROP TABLE IF EXISTS RFMDM_TEMP''')
conn.commit()
orig_retail_df.to_sql('RFMDM_TEMP',conn,if_exists = 'replace')
#rfmdm = pd.read_sql("select * from RFMDM",conn)
print ("new table got created successfully")
conn.close()

Opened database successfully


new table got created successfully


In [4]:
#check record exists in table
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cur = conn.execute("select count(*) from RFMDM_TEMP")
data = cur.fetchall()
conn.close()
print(data)

Opened database successfully
[(404909,)]


In [6]:
#clean the data
#select count(*) from RFMDM_TEMP where StockCode in ('POST', 'D', 'BANK CHARGES', 'C2', 'CRUK', 'DOT', 'M', 'PADS')

#--1920

#delete from rfmdm_temp where StockCode in ('POST', 'D', 'BANK CHARGES', 'C2', 'CRUK', 'DOT', 'M', 'PADS')

conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cur = conn.execute("select count(*) from RFMDM_TEMP where StockCode in ('POST', 'D', 'BANK CHARGES', 'C2', 'CRUK', 'DOT', 'M', 'PADS')")
data = cur.fetchall()
conn.close()
print(data)

Opened database successfully
[(1920,)]


In [7]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
conn.execute("delete from RFMDM_TEMP where StockCode in ('POST', 'D', 'BANK CHARGES', 'C2', 'CRUK', 'DOT', 'M', 'PADS')")
conn.commit()
conn.close()
print("records got deleted successfully")

Opened database successfully


records got deleted successfully


In [4]:
#create the dataframe
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
rfmdm = pd.read_sql("select * from RFMDM_TEMP",conn)
print ("records added to dataframe successfully")
conn.close()

Opened database successfully
records added to dataframe successfully


In [5]:
rfmdm.dtypes

index           int64
InvoiceNo      object
StockCode      object
Quantity        int64
DoP            object
UnitPrice     float64
CustomerID      int64
Country        object
dtype: object

In [6]:
#convert DoP to dattime object
rfmdm['DoP'] = pd.to_datetime(rfmdm['DoP'])
print("Converted successfully")

Converted successfully


In [7]:
#check data type
rfmdm.dtypes

index                  int64
InvoiceNo             object
StockCode             object
Quantity               int64
DoP           datetime64[ns]
UnitPrice            float64
CustomerID             int64
Country               object
dtype: object

In [8]:
rfmdm['Year'] = rfmdm['DoP'].dt.year
rfmdm['Month'] = rfmdm['DoP'].dt.month
rfmdm['TM_Id'] = rfmdm['Year'].astype(str) + '_' + rfmdm['Month'].astype(str)
rfmdm['Revenue'] = rfmdm['Quantity'] * rfmdm['UnitPrice']
rfmdm.tail()

,index,InvoiceNo,StockCode,Quantity,DoP,UnitPrice,CustomerID,Country,Year,Month,TM_Id,Revenue
404904,406824,581587,22613,12,2011-12-09,0.85,12680,France,2011,12,2011_12,10.20
404905,406825,581587,22899,6,2011-12-09,2.10,12680,France,2011,12,2011_12,12.60
404906,406826,581587,23254,4,2011-12-09,4.15,12680,France,2011,12,2011_12,16.60
404907,406827,581587,23255,4,2011-12-09,4.15,12680,France,2011,12,2011_12,16.60
404908,406828,581587,22138,3,2011-12-09,4.95,12680,France,2011,12,2011_12,14.85


In [13]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cursor = conn.cursor()
cursor.execute('''DROP TABLE IF EXISTS RFMDM''')
conn.commit()
rfmdm.to_sql('RFMDM',conn,if_exists = 'replace')
#rfmdm = pd.read_sql("select * from RFMDM",conn)
print ("new table got created successfully")
conn.close()

Opened database successfully


new table got created successfully


In [9]:
#check record exists in table
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cur = conn.execute("select count(*) from RFMDM")
data = cur.fetchall()
conn.close()
print(data)

Opened database successfully
[(404909,)]


In [9]:
rfmdm.describe()

,index,Quantity,UnitPrice,CustomerID,Year,Month,Revenue
count,404909.000000,404909.000000,404909.000000,404909.000000,404909.000000,404909.000000,404909.000000
mean,203436.614595,12.105799,2.901129,15295.168304,2010.933975,7.606902,20.465496
std,117452.806121,249.237543,4.430846,1710.424057,0.248326,3.418977,422.775982
min,0.000000,-80995.000000,0.000000,12346.000000,2010.000000,1.000000,-168469.600000
25%,101725.000000,2.000000,1.250000,13969.000000,2011.000000,5.000000,4.200000
50%,203452.000000,5.000000,1.950000,15159.000000,2011.000000,8.000000,10.900000
75%,305171.000000,12.000000,3.750000,16794.000000,2011.000000,11.000000,19.500000
max,406828.000000,80995.000000,649.500000,18287.000000,2011.000000,12.000000,168469.600000


In [42]:
#rfmdm_DoPIndexed = rfmdm.copy()
#rfmdm_DoPIndexed = rfmdm_DoPIndexed.set_index("DoP")
#rfmdm_DoPIndexed.head()

### Create tables based on sales transaction data for Dec'10 and Jan'11

#run following queries in sql
select * from rfmdm

select count(*) from rfmdm where TM_Id = '2010_12'
--26734

--create view for sales data in dec'2010
drop view rfmdm_dec_2010
create view rfmdm_dec_2010 as select * from rfmdm where TM_Id = '2010_12'

select count(*) from rfmdm_dec_2010
--26734


--create view for sales data in jan'2011
select count(*) from rfmdm where TM_Id = '2011_1'
--21807

drop view rfmdm_jan_2011
create view rfmdm_jan_2011 as select * from rfmdm where TM_Id = '2011_1'

select count(*) from rfmdm_jan_2011
--21807



### Create segmentation table for Dec'2010 and Jan'2011 group by Customer Id
calculate recency, frequency and monetary for each month



-- create segmentation table for dec'2010
drop table if exists RFMSEG_2010_12

CREATE TABLE RFMSEG_2010_12 (
    CustomerID  CHAR (20),
    RDate   DATE,
    Frequency    INTEGER,
    Product INTEGER,
    TDate         DATE,
    Monetary   FLOAT,
    TM_Id CHAR(20))
    
INSERT INTO RFMSEG_2010_12
       SELECT CustomerID, max(DoP), count(distinct(InvoiceNo)), count(distinct(StockCode)), min(DoP), sum(quantity*unitprice), TM_Id
       FROM rfmdm_dec_2010
       group by customerid
       
select count(*) from RFMSEG_2010_12
--947

select * from RFMSEG_2010_12


drop view if exists RFMSEG_2010_12_View

-- add recency 
--create a view for Dec'2010 customer data set (calculate Recency also)

create view RFMSEG_2010_12_View AS select (julianday('2011-01-01') - julianday(RDate)) AS Recency, CUstomerId,RDate,Frequency,Product,TDate,Monetary,TM_Id from RFMSEG_2010_12

select * from RFMSEG_2010_12_View order by Recency ASC

--------------------------------

-- create segmentation table for jan'2011
drop table if exists RFMSEG_2011_01

CREATE TABLE RFMSEG_2011_01 (
    CustomerID  CHAR (20),
    RDate   DATE,
    Frequency    INTEGER,
    Product INTEGER,
    TDate         DATE,
    Monetary   FLOAT,
    TM_Id CHAR(20))
    
select * from rfmdm_jan_2011


INSERT INTO RFMSEG_2011_01
       SELECT CustomerID, max(DoP), count(distinct(InvoiceNo)), count(distinct(StockCode)), min(DoP), sum(quantity*unitprice), TM_Id
       FROM rfmdm_jan_2011
       group by customerid
       
select count(*) from RFMSEG_2011_01
--780

select * from RFMSEG_2011_01


-- Add recency
--create a view for Jna'2011 customer data set (calculate Recency also)

drop view if exists RFMSEG_2011_01_View

create view RFMSEG_2011_01_View AS select (julianday('2011-02-01') - julianday(RDate)) AS Recency, CUstomerId,RDate,Frequency,Product,TDate,Monetary,TM_Id from RFMSEG_2011_01

select * from RFMSEG_2011_01_View order by Recency ASC

### create 3 separate views
First view will contain details for non-repeating customers in Dec'2010
Second view will contain details for repeating customers in Dec'2010 and Jan'2011
Third view will contain details for non-repeating customers in Jan'2011

drop view if exists dec_2010_non_repeat

--non repeating customers in Dec'2010

select count(*) from RFMSEG_2010_12_View A where A.CustomerId not in (select B.CustomerId from RFMSEG_2011_01_View B)

--587

create view dec_2010_non_repeat as select * from RFMSEG_2010_12_View A where A.CustomerId not in (select B.CustomerId from RFMSEG_2011_01_View B)

-- repeating customers in Dec'2010 who are available in Jan'2011 also

select count(*) from RFMSEG_2010_12_View A where A.CustomerId in (select B.CustomerId from RFMSEG_2011_01_View B)
--360

drop view if exists dec_2010_repeat

create view dec_2010_repeat as select * from RFMSEG_2010_12_View A where A.CustomerId in (select B.CustomerId from RFMSEG_2011_01_View B)

-- create view for new customers who are available only in Jan'2011 but not in Dec'2010


select count(*) from RFMSEG_2011_01_View A where A.CustomerId not in (select B.CustomerId from RFMSEG_2010_12_View B)
--420

drop view if exists jan_2011_non_repeat

create view jan_2011_non_repeat as select * from RFMSEG_2011_01_View A where A.CustomerId not in (select B.CustomerId from RFMSEG_2010_12_View B)

#Create the dataframes based on the 3 views created above

In [11]:
#create the dataframes
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")

jan_2011_all = pd.read_sql("select * from RFMSEG_2011_01_View",conn)
print ("records added to dataframe successfully")
dec_2010_all = pd.read_sql("select * from RFMSEG_2010_12_View",conn)
print ("records added to dataframe successfully")

dec_2010_non_repeat = pd.read_sql("select * from dec_2010_non_repeat",conn)
print ("records added to dataframe successfully")
dec_2010_repeat = pd.read_sql("select * from dec_2010_repeat",conn)
print ("records added to dataframe successfully")
jan_2011_non_repeat = pd.read_sql("select * from jan_2011_non_repeat",conn)
print ("records added to dataframe successfully")
conn.close()

Opened database successfully
records added to dataframe successfully
records added to dataframe successfully
records added to dataframe successfully
records added to dataframe successfully
records added to dataframe successfully


### Calculate Recency for 2 months data (Dec'2010 and Jan'2011)

In [12]:
#create copy of dec dataframe
dec_2010_non_repeat_1 = dec_2010_non_repeat.copy(deep = True)

In [13]:
#update recency for the customers who visited in Dec'2010 but did not visit in Jan'2011
print("before update")
dec_2010_non_repeat_1.head()
dec_2010_non_repeat_1['Recency'] = dec_2010_non_repeat_1['Recency'] + 31
print("after update")
dec_2010_non_repeat_1.head()

before update


,Recency,CustomerID,RDate,Frequency,Product,TDate,Monetary,TM_Id
0,15.0,12370,2010-12-17 00:00:00,2,89,2010-12-14 00:00:00,1748.02,2010_12
1,11.0,12423,2010-12-21 00:00:00,1,15,2010-12-21 00:00:00,192.93,2010_12
2,9.0,12427,2010-12-23 00:00:00,2,9,2010-12-03 00:00:00,192.55,2010_12
3,23.0,12429,2010-12-09 00:00:00,1,19,2010-12-09 00:00:00,1137.50,2010_12
4,24.0,12433,2010-12-08 00:00:00,2,79,2010-12-01 00:00:00,3787.12,2010_12


after update


,Recency,CustomerID,RDate,Frequency,Product,TDate,Monetary,TM_Id
0,46.0,12370,2010-12-17 00:00:00,2,89,2010-12-14 00:00:00,1748.02,2010_12
1,42.0,12423,2010-12-21 00:00:00,1,15,2010-12-21 00:00:00,192.93,2010_12
2,40.0,12427,2010-12-23 00:00:00,2,9,2010-12-03 00:00:00,192.55,2010_12
3,54.0,12429,2010-12-09 00:00:00,1,19,2010-12-09 00:00:00,1137.50,2010_12
4,55.0,12433,2010-12-08 00:00:00,2,79,2010-12-01 00:00:00,3787.12,2010_12


In [14]:
#merge the customers - Jan2011 + Non-Repeat customers of Dec2010
rfmseg_2010_12_2011_01_R = pd.concat([jan_2011_all,dec_2010_non_repeat_1], axis=0)
rfmseg_2010_12_2011_01_R.head()
rfmseg_2010_12_2011_01_R.describe()

,Recency,CustomerID,RDate,Frequency,Product,TDate,Monetary,TM_Id
0,14.0,12346,2011-01-18 00:00:00,2,1,2011-01-18 00:00:00,0.00,2011_1
1,6.0,12347,2011-01-26 00:00:00,1,29,2011-01-26 00:00:00,475.39,2011_1
2,7.0,12348,2011-01-25 00:00:00,1,5,2011-01-25 00:00:00,187.44,2011_1
3,14.0,12356,2011-01-18 00:00:00,1,35,2011-01-18 00:00:00,1947.62,2011_1
4,20.0,12359,2011-01-12 00:00:00,1,16,2011-01-12 00:00:00,547.50,2011_1


,Recency,Frequency,Product,Monetary
count,1367.000000,1367.000000,1367.000000,1367.000000
mean,29.975128,1.542794,24.212143,528.846386
std,20.997510,1.405190,24.937029,1518.835961
min,1.000000,1.000000,1.000000,-1241.430000
25%,10.000000,1.000000,8.000000,160.215000
50%,25.000000,1.000000,17.000000,300.650000
75%,50.000000,2.000000,32.000000,490.985000
max,62.000000,34.000000,212.000000,27834.610000


In [15]:
jan_2011_all.shape
dec_2010_non_repeat_1.shape
rfmseg_2010_12_2011_01_R.shape

(780, 8)

(587, 8)

(1367, 8)

In [16]:
#create the table
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cursor = conn.cursor()
cursor.execute('''DROP TABLE IF EXISTS RFMSEG_2010_12_2011_01_R''')
conn.commit()
rfmseg_2010_12_2011_01_R.to_sql('RFMSEG_2010_12_2011_01_R',conn,if_exists = 'replace')
print ("new table got created successfully")
conn.close()

Opened database successfully


new table got created successfully


In [17]:
rfmseg_2010_12_2011_01_R.head()

,Recency,CustomerID,RDate,Frequency,Product,TDate,Monetary,TM_Id
0,14.0,12346,2011-01-18 00:00:00,2,1,2011-01-18 00:00:00,0.00,2011_1
1,6.0,12347,2011-01-26 00:00:00,1,29,2011-01-26 00:00:00,475.39,2011_1
2,7.0,12348,2011-01-25 00:00:00,1,5,2011-01-25 00:00:00,187.44,2011_1
3,14.0,12356,2011-01-18 00:00:00,1,35,2011-01-18 00:00:00,1947.62,2011_1
4,20.0,12359,2011-01-12 00:00:00,1,16,2011-01-12 00:00:00,547.50,2011_1


In [18]:
rfmseg_2010_12_2011_01_R_1 = rfmseg_2010_12_2011_01_R.filter(['CustomerID', 'Recency'], axis = 1)

In [19]:
rfmseg_2010_12_2011_01_R_1.head()

,CustomerID,Recency
0,12346,14.0
1,12347,6.0
2,12348,7.0
3,12356,14.0
4,12359,20.0


### calculate avg monetary and frequency for 2 months (Dec'2010 and Jan'2011)

In [ ]:
#run the following queries in sql

In [ ]:
select count(*) from RFMSEG_2011_01_View
--780

select count(*) from RFMSEG_2010_12_View
--947

drop view if exists RFMSEG_2010_12_2011_01_M_F

create view RFMSEG_2010_12_2011_01_M_F AS 
select * from RFMSEG_2011_01_View
UNION ALL
select * from RFMSEG_2010_12_View

select count(*) from RFMSEG_2010_12_2011_01_M_F
--1727

select * from RFMSEG_2010_12_2011_01_M_F

drop view if exists RFMSEG_2010_12_2011_01_M_F_aggr

create view RFMSEG_2010_12_2011_01_M_F_aggr as
select CustomerID, AVG(Frequency) as AVG_FREQ, avg(Monetary) as AVG_MONETARY
from RFMSEG_2010_12_2011_01_M_F
group by CustomerID

select * from RFMSEG_2010_12_2011_01_M_F_aggr

In [ ]:
#create dataframe based on the view created above

In [20]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
rfmseg_2010_12_2011_01_M_F_aggr = pd.read_sql("select * from RFMSEG_2010_12_2011_01_M_F_aggr",conn)
print ("records added to dataframe successfully")
conn.close()

Opened database successfully
records added to dataframe successfully


In [21]:
rfmseg_2010_12_2011_01_M_F_aggr.head()

,CustomerID,AVG_FREQ,AVG_MONETARY
0,12346,2.0,0.00
1,12347,1.0,593.59
2,12348,1.0,420.12
3,12356,1.0,1947.62
4,12359,1.0,547.50


In [26]:
rfmseg_2010_12_2011_01_R_1.head()

,CustomerID,Recency
0,12346,14.0
1,12347,6.0
2,12348,7.0
3,12356,14.0
4,12359,20.0


### Create a single dataframe containing Recency, Frequency and Monetary for Dec'2010 and Jan'2011

In [27]:
merged_df = pd.merge(rfmseg_2010_12_2011_01_M_F_aggr, rfmseg_2010_12_2011_01_R_1, on = 'CustomerID')
rfmseg_2010_12_2011_01_M_F_aggr.shape
rfmseg_2010_12_2011_01_R_1.shape
merged_df.shape
merged_df.head()

(1367, 3)

(1367, 2)

(1367, 4)

,CustomerID,AVG_FREQ,AVG_MONETARY,Recency
0,12346,2.0,0.00,14.0
1,12347,1.0,593.59,6.0
2,12348,1.0,420.12,7.0
3,12356,1.0,1947.62,14.0
4,12359,1.0,547.50,20.0
